### 1. Lagrangian duality

In [105]:
#a
using JuMP, Gurobi

m = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m,x_1)
@variable(m,x_2)
@constraint(m, 1 - x_1 <= 0)
@objective(m, Min, (x_1^2 + x_2^2)/2)
status = solve(m)
println(status)
println("p* = ", getobjectivevalue(m))
println("x1 = ", getvalue(x_1))
println("x2 = ", getvalue(x_2))
;

Academic license - for non-commercial use only
Optimal
p* = 0.5
x1 = 1.0
x2 = -0.0


b) The lagrangian equation is $L(x, \lambda) = {(x_1^2+x_2^2) \over 2} + \lambda(1-x_1)$ <br>
Since the coefficients for $x_1$ and $x_2$ are all positive, it is bounded. The minimum occurs when $x_1 = \lambda$,$x_2 = 0$ <br>
<center>$g(\lambda) = -{1\over 2} \lambda^2 + \lambda$</center>


In [106]:
#c
using JuMP, Gurobi

m = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m,λ >= 0)
@objective(m, Max, (-λ^2/2 + λ))
status = solve(m)
println(status)
println("d* = ", getobjectivevalue(m))
println("λ = ", getvalue(λ))
;

Academic license - for non-commercial use only
Optimal
d* = 0.5
λ = 1.0


d) Yes the Slater condition is satisfied for this problem, since the convexity for D and f holds, and there do exists some x in the interior of D such that f(x) < 0. <br>
Obviously, in our case d* = p*, the strong duality holds.

### 2. Thrift store

In [107]:
#a
using JuMP, Cbc

m = Model(solver=CbcSolver())
@variable(m, x[1:5], Int)
for i in 1:5
    @constraint(m, x[i] >= 0)
end
@constraint(m, x[1] + 5x[2] + 10x[3] + 25x[4] + 50x[5] == 99)
@objective(m, Min, 2.5x[1] + 5x[2] + 2.268x[3] + 5.670x[4] + 11.34x[5])

status = solve(m)
println(status)
println("#penny used = ", getvalue(x[1]))
println("#nickel used = ", getvalue(x[2]))
println("#dime used = ", getvalue(x[3]))
println("#quarter used = ", getvalue(x[4]))
println("#half dollar used = ", getvalue(x[5]))
println("total weight = ", getobjectivevalue(m))
;

Optimal
#penny used = 4.0
#nickel used = -0.0
#dime used = 2.0
#quarter used = 3.0
#half dollar used = -0.0
total weight = 31.546


In [108]:
#b
using JuMP, Cbc

m = Model(solver=CbcSolver())
@variable(m, x[1:5], Int)
for i in 1:5
    @constraint(m, x[i] >= 0)
end
@constraint(m, x[1] + 5x[2] + 10x[3] + 25x[4] + 50x[5] == 99)
@constraint(m, 2.5x[1] + 5x[2] + 2.268x[3] + 5.670x[4] + 11.34x[5] <= 35)
@objective(m, Min, sum(x[i] for i in 1:5))

status = solve(m)
println("# penny used = ", getvalue(x[1]))
println("# nickel used = ", getvalue(x[2]))
println("# dime used = ", getvalue(x[3]))
println("# quarter used = ", getvalue(x[4]))
println("# half dollar used = ", getvalue(x[5]))
println("# of coins = ", getobjectivevalue(m))
;

# penny used = 4.0
# nickel used = -0.0
# dime used = 2.0
# quarter used = 1.0
# half dollar used = 1.0
# of coins = 8.0


In part b, it uses 1 half dollar to substitute for 2 quarters, this is reasonable since the objective in b is to minimize the number of coins used, regardless the total weight; although it is noticeable in this case that the weight of using 2 quarters is exactly the same as the weight of using 1 half dollar.

### 3. Comquat Computers

In [113]:
using JuMP, Cbc

m = Model(solver = CbcSolver())

@variable(m, x[1:4] >= 0,Int)
@variable(m, z[1:4], Bin)

@constraint(m, x[1] <= 10000)  # capacity
@constraint(m, x[2] <= 8000)  # capacity
@constraint(m, x[3] <= 9000)  # capacity
@constraint(m, x[4] <= 6000)  # capacity
@constraint(m, sum(x[i] for i in 1:4) <= 20000) #total computers
@constraint(m, x .<= 10000*z)                    # if x>0 then z=1

@objective(m, Max, 2500*x[1] + 1800*x[2] + 1200*x[3] + 600*x[4] - 
    9000000*z[1] - 5000000*z[2] - 3000000*z[3] - 1000000*z[4])

solve(m)

xopt = getvalue(x)
println("Produce ",xopt[1], " in plant 1")
println("Produce ",xopt[2], " in plant 2")
println("Produce ",xopt[3], " in plant 3")
println("Produce ",xopt[4], " in plant 4")
println("\$", getobjectivevalue(m)/1000000, " millions of maximum profit")

Produce 10000.0 in plant 1
Produce 8000.0 in plant 2
Produce 0.0 in plant 3
Produce 2000.0 in plant 4
$25.6 millions of maximum profit


### 4. ABC Investments

In [115]:
using JuMP, Cbc

m = Model(solver = CbcSolver())

min_invest = [3,2,9,5,12,4]
max_invest = [27,12,35,15,46,18]
@variable(m, x[1:6])
@variable(m, z[1:6], Bin)
for i in 1:6
    @constraint(m, min_invest[i]*z[i] <= x[i])  
    @constraint(m, x[i] <= max_invest[i]*z[i]) 
end
@constraint(m, x .<= 50*z)                    # if x>0 then z=1
@constraint(m,x[5] <= x[2] + x[4] + x[6])
@constraint(m,z[3] <= z[6])
@constraint(m,sum(x[i] for i in 1:6) <= 80)
@objective(m, Max, 0.13*x[1] + 0.09*x[2] + 0.17*x[3] + 0.10*x[4] + 0.22*x[5] + 0.12*x[6])

status = solve(m)
xopt = getvalue(x)
println(status)
println("Option 1: \$", xopt[1]," million")
println("Option 2: \$", xopt[2]," million")
println("Option 3: \$", xopt[3]," million")
println("Option 4: \$", xopt[4]," million")
println("Option 5: \$", xopt[5]," million")
println("Option 6: \$", xopt[6]," million")
println("maximum expected return: \$", getobjectivevalue(m), " million")
;

Optimal
Option 1: $0.0 million
Option 2: $0.0 million
Option 3: $34.99999999999999 million
Option 4: $5.0 million
Option 5: $22.500000000000004 million
Option 6: $17.500000000000004 million
maximum expected return: $13.500000000000004 million
